In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
import random
import re
import requests
from bs4 import BeautifulSoup
import re
import http.client
import json

In [2]:
# 读取美股kol_list
a_kol_df = pd.read_excel('a_kol.xlsx')
a_kol_df.head(5)
a_kol_list = a_kol_df['Twitter Handle'].tolist()
a_kol_list = [account.lstrip('@') for account in a_kol_list]
a_kol_list

['TheEconomist',
 'Reuters',
 'WSJ',
 'Forbes',
 'business',
 'business',
 'FinancialTimes',
 'FT',
 'CNBC',
 'CNBC',
 'BusinessInsider',
 'FortuneMagazine',
 'ZeroHedge',
 'unusual_whales',
 'YahooFinance',
 'BillAckman',
 'TheRoaringKitty',
 'WallStreetSilv',
 'michaeljburry',
 'StockTwits',
 'themotleyfool',
 'TheStreet',
 'stlouisfed',
 'DeItaone',
 'TaraBull808',
 'Carl_C_Icahn',
 'ParikPatelCFA',
 'PelosiTracker_',
 'charliebilello',
 'PeterLBrandt',
 'StockMKTNewz',
 'RaoulGMI',
 'gurgavin',
 'FinancialReview',
 'morganhousel',
 'IBDinvestors',
 'barronsonline',
 'WOLF_Financial',
 'Benzinga',
 'FXStreetNews',
 'Jake__Wujastyk',
 'SeekingAlpha',
 'LynAldenContact',
 'TradesTrey',
 'AswathDamodaran',
 'LynAldenContact',
 'MorningStarInc',
 'allstarcharts',
 'matt_levine',
 'WallStCynic',
 'BarChart',
 'ReformedBroker',
 'QuiverQuant',
 'michaelbatnick',
 'TruthGundlach',
 'litcapital',
 'ritholtz',
 'DoombergT',
 'EpsilonTheory',
 'fundstrat',
 'TheStalwart',
 'zachxbt',
 'Cramer

In [3]:


def get_user_rest_id(user_name):
    conn = http.client.HTTPSConnection("api.apidance.pro")
    payload = ''
    headers = {
    'apikey': 'q4fa83ok43io70najdgmijdt2s6fkl'
    }
    conn.request("GET", f"/graphql/UserByScreenName?variables=%7B%22screen_name%22:%22{user_name}%22,%22withSafetyModeUserFields%22:true,%22withHighlightedLabel%22:true%7D", payload, headers)
    res = conn.getresponse()
    data = res.read()
    json_data = json.loads(data.decode("utf-8"))
    rest_id = json_data['data']['user']['result']['rest_id']
    return rest_id 

def get_user_tweets_page(user_id, cursor=None):
    """获取单页用户推文"""
    conn = http.client.HTTPSConnection("api.apidance.pro")
    headers = {
       'apikey': 'q4fa83ok43io70najdgmijdt2s6fkl'
    }
    
    # 构建URL
    url = f"/sapi/UserTweets?user_id={user_id}"
    if cursor:
        url += f"&cursor={cursor}"
    else:
        url += "&cursor=null"
    
    conn.request("GET", url, '', headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))


def fetch_user_tweets(user_id, max_pages=10, save_to_file=True, filename=None):
    """
    获取用户的多页推文
    
    参数:
        user_id (str): Twitter用户ID
        max_pages (int): 最多获取的页数，默认10页
        save_to_file (bool): 是否保存到文件，默认True
        filename (str): 保存的文件名，默认为 'tweets_{user_id}.json'
    
    返回:
        list: 所有推文的列表
    """
    all_tweets = []
    cursor = None
    page = 1
    
    print(f"\n开始获取用户 {user_id} 的推文...")
    print(f"最多获取 {max_pages} 页\n")
    
    while page <= max_pages:
        print(f"{'='*60}")
        print(f"正在获取第 {page} 页...")
        print(f"{'='*60}")
        
        try:
            # 获取当前页
            response = get_user_tweets_page(user_id, cursor)
            
            # 提取推文
            tweets = response.get('tweets', [])
            all_tweets.extend(tweets)
            
            print(f"✓ 本页获取 {len(tweets)} 条推文")
            print(f"✓ 累计获取 {len(all_tweets)} 条推文")
            
            # 获取下一页游标
            next_cursor = response.get('next_cursor_str')
            
            if not next_cursor:
                print("\n✓ 没有更多推文了")
                break
            
            print(f"✓ 下一页游标: {next_cursor[:50]}...")
            
            cursor = next_cursor
            page += 1
            
            # 暂停1秒，避免请求过快
            # if page <= max_pages:
            #     time.sleep(1)
                
        except Exception as e:
            print(f"\n✗ 获取第 {page} 页时出错: {str(e)}")
            break
    
    print(f"\n{'='*60}")
    print(f"总共获取 {len(all_tweets)} 条推文")
    print(f"{'='*60}")
    
    # # 保存到文件
    # if save_to_file:
    #     if filename is None:
    #         filename = f'tweets_{user_id}.json'
        
    #     with open(filename, 'w', encoding='utf-8') as f:
    #         json.dump(all_tweets, f, indent=2, ensure_ascii=False)
        
    #     print(f"✓ 推文已保存到 {filename}")
    
    return all_tweets

In [4]:
a_kol_list[:10]

['TheEconomist',
 'Reuters',
 'WSJ',
 'Forbes',
 'business',
 'business',
 'FinancialTimes',
 'FT',
 'CNBC',
 'CNBC']

In [ ]:
for user in a_kol_list[:10]:
    user_id = get_user_rest_id(user)
    tweets = fetch_user_tweets(user_id,max_pages=20)
    filename = f'I:\\finance-agent\\X\\tweets_json\\{user}.json'
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(tweets, f, indent=2, ensure_ascii=False)
        


开始获取用户 5988062 的推文...
最多获取 20 页

正在获取第 1 页...
✓ 本页获取 20 条推文
✓ 累计获取 20 条推文
✓ 下一页游标: DAAHCgABG52eIDE__-oLAAIAAAATMTk4OTgzMDg4NTAzODg0Nj...
正在获取第 2 页...
✓ 本页获取 20 条推文
✓ 累计获取 40 条推文
✓ 下一页游标: DAAHCgABG52eIDE__9ULAAIAAAATMTk4OTczMzkwMzk2ODM5OT...
正在获取第 3 页...
✓ 本页获取 20 条推文
✓ 累计获取 60 条推文
✓ 下一页游标: DAAHCgABG52eIDE__8ALAAIAAAATMTk4OTY1NDYzNzMwNjE3MT...
正在获取第 4 页...
✓ 本页获取 20 条推文
✓ 累计获取 80 条推文
✓ 下一页游标: DAAHCgABG52eIDE__6sLAAIAAAATMTk4OTU1OTA5NjM0NTgyNT...
正在获取第 5 页...
✓ 本页获取 20 条推文
✓ 累计获取 100 条推文
✓ 下一页游标: DAAHCgABG52eIDE__5YLAAIAAAATMTk4OTQ1ODM0ODExMDk0Mj...
正在获取第 6 页...
✓ 本页获取 20 条推文
✓ 累计获取 120 条推文
✓ 下一页游标: DAAHCgABG52eIDE__4ELAAIAAAATMTk4OTM5OTIwNzk5MjU3OD...
正在获取第 7 页...
✓ 本页获取 20 条推文
✓ 累计获取 140 条推文
✓ 下一页游标: DAAHCgABG52eIDE__2wLAAIAAAATMTk4OTM1NTIwNTY3OTMwND...
正在获取第 8 页...
✓ 本页获取 20 条推文
✓ 累计获取 160 条推文
✓ 下一页游标: DAAHCgABG52eIDE__1cLAAIAAAATMTk4OTI4MjE4MTc5ODA0MD...
正在获取第 9 页...
✓ 本页获取 19 条推文
✓ 累计获取 179 条推文
✓ 下一页游标: DAAHCgABG52eIDE__0MLAAIAAAATMTk4OTE5MTU5Mjk3NzAxND...
正在获取第 10 页...
✓ 本页获取 20 条